# Efficient Pandas
### Lobdata

## Objetivos
- Parte 1: Selecionando colunas e linhas de forma eficiente.
- Parte 2: Substituindo valores em colunas do DataFrame.
- Parte 3: Iterando no dataset de forma eficiente.
- Parte 4: Manipulação de dados com o `.groupby`, `.transform` e `.filter`.

Usaremos a Base de dados das Olimpíadas de Tokyo 2021 [Link](https://www.kaggle.com/arjunprasadsarkhel/2021-olympics-in-tokyo)



In [ ]:
!pip install pyjanitor
!pip install xlrd
!pip install openpyxl

In [3]:
import pandas as pd
import numpy as np
import time
from janitor import clean_names

In [9]:
#### Importando e Limpando os dados

medals_df = pd.read_excel("Tokyo2021_Kaggle/Medals.xlsx").clean_names()

medals_df.head()

C:\Users\Danilo\AppData\Local\Programs\Python\Python39\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,rank,team_noc,gold,silver,bronze,total,rank_by_total
0,1,United States of America,39,41,33,113,1
1,2,People's Republic of China,38,32,18,88,2
2,3,Japan,27,14,17,58,5
3,4,Great Britain,22,21,22,65,4
4,5,ROC,20,28,23,71,3


# Alterando os dados após importação

- `.drop`: removi algumas colunas que não são necessárias nesse momento.
- `.clean_names`: é um método do pyjanitor, que limpa o nome das colunas automaticamente.
- `.melt`: é um pivoteamento nos dados para o formato longer, melhor para análise.


In [10]:
#### Arrumando os dados 

medals_df.drop(["total", "rank_by_total"], axis=1, inplace=True)
medals_df.head()

,rank,team_noc,gold,silver,bronze
0,1,United States of America,39,41,33
1,2,People's Republic of China,38,32,18
2,3,Japan,27,14,17
3,4,Great Britain,22,21,22
4,5,ROC,20,28,23


In [11]:
medals_reshaped_df = medals_df.melt(
    id_vars=["rank", "team_noc"],
    value_vars=["gold", "silver", "bronze"],
    var_name="medals",
    value_name="qtt",
)
medals_reshaped_df.head()

,rank,team_noc,medals,qtt
0,1,United States of America,gold,39
1,2,People's Republic of China,gold,38
2,3,Japan,gold,27
3,4,Great Britain,gold,22
4,5,ROC,gold,20


# Função para auxiliar o exercício

## A função será para fazer comparações entre os métodos

A função utiliza-se de um input onde inserimos os tempos gastos em cada método

In [14]:
# helper function to calculate pct diff between the times
def calculate_how_much_diff(method_1: float, method_2: float) -> str:
    diff = method_2 / method_1

    if diff > 1:
        ans = f"Método 1 é {round(diff*100, 2)}% vezes mais rápido que o Método 2"
    elif diff < 1:
        new_diff = method_1 / method_2
        ans = f"Método 2 é {round(new_diff*100, 2)}% vezes mais rápido que o Método 1"

    return ans

# Parte I: Selecionando colunas e linhas de forma eficiente

## .iloc e .loc

Essa é uma tarefa bem comum utilizada com `DataFrames` do Pandas, muitas vezes precisamos selecionar apenas uma porção dos dados, seja por colunas, seja por linhas. É ae que entra a utilização dos métodos `.iloc` e `.loc`.

- `.iloc`: Localizador numérico
- `.loc`: Localizador nominal

In [15]:
# .loc time

start_time = time.time()
medals_reshaped_df.loc[:, ["team_noc", "medals", "qtt"]]
delta_loc = time.time() - start_time

# .iloc time

start_time = time.time()
medals_reshaped_df.iloc[:, [1, 2, 3]]
delta_iloc = time.time() - start_time

# who is faster?
calculate_how_much_diff(method_1=delta_loc, method_2=delta_iloc)

'Método 2 é 101.78% vezes mais rápido que o Método 1'

## O método `.iloc` é mais eficiente tanto para colunas quanto para linhas

# AMOSTRAGEM ALEATÓRIA

- utilizar o módulo do numpy de geração de números randômicos e aplicar ao `DataFrame`
- utilizar o método `.sample` do Pandas

In [46]:
### Como pegar valores aleatórios em um intervalo de linhas determinado por low e high

medals_reshaped_df.iloc[np.random.randint(low=0, high=medals_reshaped_df.shape[0], size=10), :]


,rank,team_noc,medals,qtt
87,86,Côte d'Ivoire,gold,0
204,19,Kenya,bronze,2
153,59,Latvia,silver,0
80,77,Namibia,gold,0
246,59,Latvia,bronze,1
161,69,Armenia,silver,2
132,39,Israel,silver,0
198,13,New Zealand,bronze,7
246,59,Latvia,bronze,1
258,72,San Marino,bronze,2


In [45]:
medals_reshaped_df.sample(10, axis=0)

,rank,team_noc,medals,qtt
78,77,Lithuania,gold,0
186,1,United States of America,bronze,33
197,12,Brazil,bronze,8
3,4,Great Britain,gold,22
272,86,Burkina Faso,bronze,1
253,68,Dominican Republic,bronze,2
252,67,Azerbaijan,bronze,4
241,56,Ethiopia,bronze,2
145,53,Austria,silver,1
185,86,Syrian Arab Republic,silver,0


In [47]:
### Testando a velocidade entre os dois métodos de selecionar aleatóriamente

# numpy random integers
start_time = time.time()
medals_reshaped_df.iloc[np.random.randint(low=0, high=medals_reshaped_df.shape[0], size=50), :]
delta_numpy = time.time() - start_time

# using .sample
start_time = time.time()
medals_reshaped_df.sample(50, axis=0)
delta_sample = time.time() - start_time

# who is faster?
calculate_how_much_diff(method_1=delta_numpy, method_2=delta_sample)

'Método 2 é 199.52% vezes mais rápido que o Método 1'

## Com isso nós finalizamos a Parte I dessa série de tutoriais, e em resumo temos o seguinte:

- `.iloc` é mais rápido que `.loc`
- `.iloc` é mais rápido que selecionar diretamente as colunas
- `.sample` é mais rápido utilizar numpy para gerar um array de números aleatórios

# Parte II: Substituindo valores em colunas do dataset

## Substituindo valores únicos utilizando o `.replace`

In [48]:
# pure Pandas
medals_reshaped_replace_pd_df = medals_reshaped_df.copy()
start_time = time.time()
medals_reshaped_replace_pd_df.medals.loc[medals_reshaped_replace_pd_df.medals=="gold"] = 'Gold'
delta_pure_pandas = time.time() - start_time

# using .replace
medals_reshaped_replace_df = medals_reshaped_df.copy()
start_time = time.time()
medals_reshaped_replace_df["medals"].replace("gold", "Gold")  # adjusting the axis = 1 for columns
delta_replace = time.time() - start_time

# who is faster?
calculate_how_much_diff(method_1=delta_pure_pandas, method_2=delta_replace)

C:\Users\Danilo\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


'Método 2 é 6494.43% vezes mais rápido que o Método 1'

## Substituindo múltiplos valores com listas e dicionários


In [66]:
#### Substituindo valores usando operações puras do Pandas

medals_reshaped_replace_pd_df = medals_reshaped_df.copy()

medals_reshaped_replace_pd_df["rank"].loc[
    (medals_reshaped_replace_pd_df["rank"] == 1)
    | (medals_reshaped_replace_pd_df["rank"] == 2)
    | (medals_reshaped_replace_pd_df["rank"] == 3)
] = "TOP 3"

medals_reshaped_replace_pd_df["rank"].loc[
    (medals_reshaped_replace_pd_df["rank"] != "TOP 3")
] = "OUTRAS"

medals_reshaped_replace_pd_df.loc[1:10,"rank"]

1      TOP 3
2      TOP 3
3     OUTRAS
4     OUTRAS
5     OUTRAS
6     OUTRAS
7     OUTRAS
8     OUTRAS
9     OUTRAS
10    OUTRAS
Name: rank, dtype: object

In [63]:
### Substituindo usando pandas, .replace() e listas []

medals_reshaped_replace_df = medals_reshaped_df.copy()

max_rank = medals_reshaped_replace_df["rank"].max()
medals_reshaped_replace_df["rank"].replace(
    [[1, 2, 3], [range(4, max_rank + 1)]], ["TOP 3", "OUTRAS"], inplace=True
)
medals_reshaped_replace_df.loc[1:10,"rank"]

1      TOP 3
2      TOP 3
3     OUTRAS
4     OUTRAS
5     OUTRAS
6     OUTRAS
7     OUTRAS
8     OUTRAS
9     OUTRAS
10    OUTRAS
Name: rank, dtype: object

In [70]:
### Substituindo valores utilizando Pandas .replace() e Dicionários {}

medals_reshaped_replace_df = medals_reshaped_df.copy()

medals_reshaped_replace_df["medals"].replace(
    {"gold": "Gold", "silver": "Silver", "bronze": "Bronze"}, inplace=True
)

medals_reshaped_replace_df.loc[:10, "medals"]

0     Gold
1     Gold
2     Gold
3     Gold
4     Gold
5     Gold
6     Gold
7     Gold
8     Gold
9     Gold
10    Gold
Name: medals, dtype: object

# FIM